In [ ]:
from ultralytics import YOLO
import torch
import torch.nn.utils.prune as prune
import os

MODEL_PATH = "./best.pt"  

def apply_pruning(model, amount=0.3):
    print(f"Applying Pruning : (Ratio : {amount*100}%)")
    
    count = 0
    for name, module in model.model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
            count += 1
            
    print(f"{count} layers applied!")

def main():
    # 1. 이미 똑똑해진 모델 불러오기
    print(f"Loading model : {MODEL_PATH}")
    if not os.path.exists(MODEL_PATH):
        print("Error")
        return

    model = YOLO(MODEL_PATH)

    apply_pruning(model, amount=0.25)

    print("Applying Quantization and exporting onnx")
    
    success = model.export(
        format='onnx', 
        dynamic=False, 
        imgsz=128,   
        half=True,   
        opset=11,
        device=1
    )

    print("-" * 30)
    print(f"Exportation complete : {success}")

if __name__ == '__main__':
    main()

Loading model : ./best.pt
Applying Pruning : (Ratio : 25.0%)
64 layers applied!
Applying Quantization and exporting onnx
Ultralytics 8.3.235  Python-3.12.10 torch-2.10.0a0+rocm7.10.0a20251106 CUDA:1 (AMD Radeon RX 7800 XT, 16368MiB)
Model summary (fused): 72 layers, 4,765,733 parameters, 0 gradients

PyTorch: starting from 'best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 4807, 336) (18.5 MB)

ONNX: starting export with onnx 1.19.1 opset 11...
ONNX: slimming with onnxslim 0.1.78...
ONNX: export success  7.1s, saved as 'best.onnx' (9.2 MB)

Export complete (10.7s)
Results saved to C:\Users\garli\Downloads\yolo
Predict:         yolo predict task=detect model=best.onnx imgsz=128 half 
Validate:        yolo val task=detect model=best.onnx imgsz=128 data=data.yaml half 
Visualize:       https://netron.app
------------------------------
Exportation complete : best.onnx
